# Библиотеки

In [1]:
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 555.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [23]:
!pip install numpy==1.20 -q   #  -q   # ==1.23 -q
!pip install pyLDAvis -q
import numpy
import pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [11]:
!pip install geopy -q
!pip install nltk -q
!pip install pandas ==1.5.3 -q

ERROR: Invalid requirement: '==1.5.3'


In [12]:
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy import Yandex     # на будущее, чтобы ошибки исправлять
from geopy import BANFrance  # только франция
import numpy as np
import time

#import pymorphy2
# import nltk
from nltk.corpus import stopwords
import re
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [19]:
import numpy as np
import pyLDAvis
import pyLDAvis.lda_model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Авалон/Диплом')

In [ ]:
nltk.download('stopwords')

NameError: ignored

# Данные

## Загрузка

In [ ]:
df = pd.read_csv('data/data.csv')
df['rating'] = df['rating'].astype('float16')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Создание полезных словарей

In [ ]:
# для подсчета количества фирм в каждой рубрике
# составление словаря рубрика - компании в этой рубрике
rubric_set_firm = defaultdict(set)
for rub_all_x, name_x in zip(df['rubrics'], df['name_ru']):
    for rub_x in set(rub_all_x.split(";")):
        rubric_set_firm[rub_x].add(name_x)

df_tmp = pd.DataFrame(rubric_set_firm.items())
df_tmp.columns = ['Рубрика', 'Кол-во фирм']
df_tmp['Кол-во фирм'] = df_tmp['Кол-во фирм'].apply(lambda x: len(x))
df_tmp = df_tmp.set_index('Кол-во фирм').sort_index(ascending=False)
df_tmp.head(10)


In [ ]:
# для подсчета количества отзывоа по каждой рубрике
# составление словаря рубрика: [фирма, фирма, ..., фирма]
rubric_name_list = defaultdict(list)
for rub_all_x, name_x in zip(df['rubrics'], df['name_ru']):
    for rub_x in set(rub_all_x.split(";")):
        rubric_name_list[rub_x].append(name_x)

df_tmp = pd.DataFrame(rubric_name_list.items())
df_tmp.columns = ['Рубрика', 'Кол-во отзывов']
df_tmp['Кол-во отзывов'] = df_tmp['Кол-во отзывов'].apply(lambda x: len(x))
df_tmp = df_tmp.set_index('Кол-во отзывов').sort_index(ascending=False)
df_tmp.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Рубрика
Кол-во отзывов,
58496,Кафе
56761,Ресторан
43133,Гостиница
21346,Магазин продуктов
19746,Супермаркет
19238,"Бар, паб"
17910,Салон красоты
16859,Быстрое питание
14394,Доставка еды и обедов


In [ ]:
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee.index = np.arange(coffee.shape[0])
coffee.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 117818 entries, 0 to 117817
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   address  117818 non-null  object 
 1   name_ru  117587 non-null  object 
 2   rating   117818 non-null  float16
 3   rubrics  117818 non-null  object 
 4   text     117818 non-null  object 
dtypes: float16(1), object(4)
memory usage: 4.7+ MB


## Получение координат

### Блок для экспериментов

In [ ]:
coffee.loc[:N, 'address'].apply(get_coords)

,0,place_id,licence,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,boundingbox
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,213409685.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,1.417989e+09,47.2316881,39.6248414,amenity,post_office,30.0,0.00001,amenity,Ростов-на-Дону 344015,"Ростов-на-Дону 344015, 54, улица Ерёменко, ЗЖМ...","[47.2316381, 47.2317381, 39.6247914, 39.6248914]"
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,181562143.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,1.215826e+08,59.8787113,30.447693,highway,residential,26.0,0.10001,road,переулок Матюшенко,"переулок Матюшенко, Щемиловка, Ивановский окру...","[59.8784221, 59.8789814, 30.4467917, 30.4485239]"
29,NaN,181562143.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,1.215826e+08,59.8787113,30.447693,highway,residential,26.0,0.10001,road,переулок Матюшенко,"переулок Матюшенко, Щемиловка, Ивановский окру...","[59.8784221, 59.8789814, 30.4467917, 30.4485239]"
42,NaN,205602885.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,4.999280e+08,55.7366859,36.8685566,highway,residential,26.0,0.10001,road,Парковая улица,"Парковая улица, Звенигород, Одинцовский городс...","[55.7356867, 55.7373080, 36.8681578, 36.8685566]"
43,NaN,205602885.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,4.999280e+08,55.7366859,36.8685566,highway,residential,26.0,0.10001,road,Парковая улица,"Парковая улица, Звенигород, Одинцовский городс...","[55.7356867, 55.7373080, 36.8681578, 36.8685566]"
76,NaN,204674065.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,2.516073e+09,55.6044778,38.0822717,leisure,sports_centre,30.0,0.00001,leisure,"Картинг центр ""200 МЕТРОВ""","Картинг центр ""200 МЕТРОВ"", 2А, улица Баженова...","[55.6044278, 55.6045278, 38.0822217, 38.0823217]"


### Извлечение координат из данных

In [ ]:
# Функция для извлечения дат
geolocator = Nominatim(user_agent="Tester")
def get_coords(address_x):
  try:
    location = geolocator.geocode(address_x, timeout=10)
  except:
    print(address_x)
    return pd.Series([None])
  if location is None:
    # из 15/100 пропусков
    list_address = address_x.split(',')
    mask = pd.Series(['корп.' not in  i for i in list_address])
    new_address = ','.join(pd.Series(list_address)[mask].to_list())
    location = geolocator.geocode(new_address, timeout=10)
  if location is None:
    return pd.Series([None])

  return pd.Series(location.raw)


In [ ]:
# извлечение координат из первых N элементов
N = 100
start = time.time()
сооrd = coffee.loc[:N-1, 'address'].apply(get_coords)[['osm_type', 'lat', 'lon', 'class', 'type', 'place_rank', 'importance', 'addresstype']]
end = time.time()

print(f"Время выполнения --  {round(end - start, 2)} секунд")
print(f"Количество пропусков в {N} элементах -- {сооrd['lat'].isna().sum()} / {сооrd.shape[0]}")

Время выполнения --  65.23 секунд
Количество пропусков в 100 элементах -- 17 / 100


In [ ]:
# солияние df и сооrd
df_tmp = coffee.loc[:сооrd.shape[0]]
df_new = pd.concat([df_tmp, сооrd], axis=1)
df_new.head()

,address,name_ru,rating,rubrics,text,osm_type,lat,lon,class,type,place_rank,importance,addresstype
0,"Воронежская область, Богучарский район, М-4 До...",У тещи,4.0,Кафе,Глубинка страны во всех своих проявлениях. Асс...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Москва, 4-й Кожевнический переулок, 4",Jinju,5.0,Кафе;Кофейня,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",way,55.725156150000004,37.6491671,building,yes,30.0,0.00001,building
2,"Москва, 4-й Кожевнический переулок, 4",Jinju,4.0,Кафе;Кофейня,"Не очень удобное расположение, от метро идти м...",way,55.725156150000004,37.6491671,building,yes,30.0,0.00001,building
3,"Краснодарский край, городской округ Сочи, посё...",Пандок,2.0,Ресторан,"Самый большой плюс это месторасположение, набе...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Краснодарский край, городской округ Сочи, посё...",Пандок,5.0,Ресторан,Добрый день! Сегодня во второй раз посетили с ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# вывод проблемных адресов
df_new[df_new['lat'].isna()]['address'].loc[0]

'Воронежская область, Богучарский район, М-4 Дон, 770-й километр'

# Предобработка отзывов

## Блок для экспериментов

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# приведения слова к нормальной форме
morph.parse('мониторы')[0].normal_form

'монитор'

In [ ]:
# стоп слова
stopwords.words("russian")[:4]

['и', 'в', 'во', 'не']

## Обработка

### функции

In [ ]:
def preprocess_text(data, stop_words):
    """Очистка текста и удаление стоп-слов"""
    # замена ё на е // оставляем буквы, пробелы и знаки препинания // удаление пробелов
    text = re.sub('ё', 'е', data.lower())
    text = re.sub(r"([.,!?])", r'\1', text)
    text =re.sub(r"[^а-яА-Я\s]+", r'', text)
    text = text.strip()

    # удаление стоп слов и слов меньше длины 3
    stop_words.extend(['очень', 'самый', 'самое', 'самая'])
    text = [w for w in text.split() if w not in stop_words]
    text = [w for w in text if len(w) >= 3]

    return ' '.join(text)

def lemmatization_text(data, morph):
    result = " ".join([morph.parse(word_x)[0].normal_form
                      for word_x in data.split()])
    return result

def get_result(data, morph=pymorphy2.MorphAnalyzer(), stop_words=stopwords.words("russian")):
    if data is not np.NaN:
        result = preprocess_text(data=data, stop_words=stop_words)
        result = lemmatization_text(data=result, morph=morph)
        return result
    return ""

def transform_data(data: pd.Series) -> list:
  transform_lst = [get_result(data=i) for i in data]
  transform_lst = [x for x in transform_lst if x not in [None, '']]
  return transform_lst


NameError: ignored

### Преобразование отзывов к норм виду + CountVectorizer

In [ ]:
pyLDAvis.enable_notebook()

def theme_class(input_data, vectorizer_x=None, model_type=None):
  result = transform_data(input_data)

  if vectorizer_x is None:
    vectorizer_x = CountVectorizer(analyzer='word',
                                 min_df=10,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))

  res_vectorizer = vectorizer_x.fit_transform(result)

  if model_type is None:
    model_type = LatentDirichletAllocation(n_components=5,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)
  model_type.fit(res_vectorizer)


  panel = pyLDAvis.lda_model.prepare(model_type,
                                     res_vectorizer,
                                     vectorizer,
                                     mds='tsne')
  print("Log Likelihood", model_type.score(res_vectorizer))
  print("Perplexity", model_type.perplexity(res_vectorizer))

  return panel, model_type, vectorizer_x

def show_topics( vectorizer_x=None, model=None, n_words=20):
  feature_names = np.array(vectorizer_x.get_feature_names_out())
  top_words = []

  for topic_weights in model.components_:
    top_keywords_locs = (-topic_weights).argsort()[:n_words]
    top_words.append(feature_names.take(top_keywords_locs))
  return top_words

NameError: ignored

In [ ]:
start = time.time()
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee.index = np.arange(coffee.shape[0])

vectorizer = TfidfVectorizer(analyzer='word',
# vectorizer = CountVectorizer(analyzer='word',
                             #binary=True,    # позволяет кодировать не по частоте, а просто по наличию (есть/нет слова)
                                 min_df=3,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))


LDA_model = LatentDirichletAllocation(n_components=6,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)


N = 1000
panel_x, LDA_model, vectorizer = theme_class(coffee.loc[:N-1, 'text'],
                                             vectorizer_x=vectorizer,
                                             model_type=LDA_model)
print(f'Время обработки = {time.time() - start} секунд')
panel_x

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


Log Likelihood -12094.335642920654
Perplexity 2427.736536368712
Время обработки = 14.446062326431274 секунд


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4     -14.366463 -36.429626       1        1  19.351713
3      11.828945  24.215630       2        1  19.192249
0      68.577057  57.887718       3        1  17.956549
5     -53.784851  30.278816       4        1  17.163130
2      -2.718319  88.517807       5        1  13.547236
1      61.294880 -19.304926       6        1  12.789123, topic_info=                       Term       Freq      Total Category  logprob  loglift
44        вежливый персонал   9.000000   9.000000  Default  30.0000  30.0000
79              вкусный еда  12.000000  12.000000  Default  29.0000  29.0000
478           самый вкусный   9.000000   9.000000  Default  28.0000  28.0000
539            уютный место   6.000000   6.000000  Default  27.0000  27.0000
359       персонал вежливый   6.000000   6.000000  Default  26.0000  26.0000
..                      ...        ...        ...      ...      ...      ...
467  рекомендовать посетить   2.943893   5.038955   Topic6  -4.2107   1.5191
451            просто огонь   1.619730   2.579137   Topic6  -4.8082   1.5914
35            быстро вкусно   2.402049   4.722704   Topic6  -4.4141   1.3805
57                 вид море   1.908371   4.457596   Topic6  -4.6442   1.2082
3        атмосфера приятный   1.758938   3.781854   Topic6  -4.7258   1.2911

[273 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
2         4  0.560140  ассортимент напиток
3         4  0.264421   атмосфера приятный
3         6  0.528841   атмосфера приятный
5         3  0.493756    атмосфера хороший
10        2  0.927766       банкетный меню
...     ...       ...                  ...
597       4  0.151382            это место
600       2  0.192289           это просто
600       5  0.769156           это просто
601       6  0.514375         это ресторан
602       2  0.728428            это самый

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 6, 3, 2])

In [ ]:
vectorizer = CountVectorizer(analyzer='word',
                                 min_df=3,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))


LDA_model = LatentDirichletAllocation(n_components=6,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)


result = transform_data(coffee.loc[:N-1, 'text'])
res_vectorizer = vectorizer.fit_transform(result)
LDA_model.fit(res_vectorizer)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-69-758530e30571>", line 2, in <cell line: 2>
    res_vectorizer = vectorizer.fit_transform(result)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 1388, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents, self.fixed_vocabulary_)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 1294, in _count_vocab
    raise ValueError(
ValueError: empty vocabulary; perhaps the documents only contain stop words

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' o

In [ ]:
pd.DataFrame(show_topics(vectorizer, LDA_model, 10))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
0,данный заведение,персонал вежливый,детский комната,еда вкусный,провести время,прекрасный обслуживание,это место,вкусный еда,выше всякий,отличный обслуживание
1,год назад,это просто,персонал приветливый,атмосфера приятный,приятно удивить,хотеть отметить,центр город,рекомендовать посетить,пальчик облизать,заказать блюдо
2,вежливый персонал,день рождение,уютный место,еда вкусный,отмечать день,приемлемый цена,выше среднее,кухня хороший,отмечать день рождение,хороший уютный
3,живой музыка,отличный место,вкусный еда,вкусный кухня,выбор блюдо,самый хороший,хороший место,большой выбор,спасибо большой,свой дело
4,высокий уровень,самый вкусный,вкусный кофе,рекомендовать посещение,спасибо огромный,вкусный еда,обслуживание вкусный,вкусно быстро,обязательно прийти,вкусно готовить
5,вкусный блюдо,подача блюдо,большой выбор,это заведение,отдельный спасибо,уютный атмосфера,приветливый персонал,хороший заведение,приятный персонал,быстрый подача
